In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Udachnyy,66.4167,112.4000,-4.77,100,94,2.04,RU,2021-04-20 20:19:16
1,1,Shingu,33.7333,135.9833,53.67,64,52,6.35,JP,2021-04-20 20:19:16
2,2,Ushuaia,-54.8000,-68.3000,50.00,57,0,4.97,AR,2021-04-20 20:19:16
3,3,Mataura,-46.1927,168.8643,51.01,83,8,4.00,NZ,2021-04-20 20:19:17
4,4,Saskylakh,71.9167,114.0833,-18.02,100,85,2.19,RU,2021-04-20 20:18:46


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,Kapaa,22.0752,-159.3190,77.00,78,75,6.91,US,2021-04-20 20:18:05
14,14,Hilo,19.7297,-155.0900,80.60,57,75,11.50,US,2021-04-20 20:19:20
17,17,Pontianak,-0.0333,109.3333,75.20,100,1,2.30,ID,2021-04-20 20:19:21
22,22,Atuona,-9.8000,-139.0333,80.51,72,3,19.04,PF,2021-04-20 20:15:55
23,23,Saint-Philippe,-21.3585,55.7679,75.20,88,90,11.50,RE,2021-04-20 20:18:08
25,25,Ransang,8.9603,117.5828,78.89,84,100,8.05,PH,2021-04-20 20:19:23
27,27,Diffa,13.3154,12.6113,89.35,5,0,6.40,NE,2021-04-20 20:19:23
28,28,Rikitea,-23.1203,-134.9692,77.49,73,59,5.28,PF,2021-04-20 20:19:24
33,33,Cabo San Lucas,22.8909,-109.9124,82.99,44,1,9.22,MX,2021-04-20 20:17:13
37,37,Valera,9.3178,-70.6036,86.97,48,46,4.00,VE,2021-04-20 20:19:26


In [10]:
preferred_cities_df.count()

City_ID       170
City          170
Lat           170
Lng           170
Max Temp      170
Humidity      170
Cloudiness    170
Wind Speed    170
Country       170
Date          170
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Kapaa,US,77.00,22.0752,-159.3190,
14,Hilo,US,80.60,19.7297,-155.0900,
17,Pontianak,ID,75.20,-0.0333,109.3333,
22,Atuona,PF,80.51,-9.8000,-139.0333,
23,Saint-Philippe,RE,75.20,-21.3585,55.7679,
25,Ransang,PH,78.89,8.9603,117.5828,
27,Diffa,NE,89.35,13.3154,12.6113,
28,Rikitea,PF,77.49,-23.1203,-134.9692,
33,Cabo San Lucas,MX,82.99,22.8909,-109.9124,
37,Valera,VE,86.97,9.3178,-70.6036,


In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Kapaa,US,77.00,22.0752,-159.3190,
14,Hilo,US,80.60,19.7297,-155.0900,
17,Pontianak,ID,75.20,-0.0333,109.3333,
22,Atuona,PF,80.51,-9.8000,-139.0333,
23,Saint-Philippe,RE,75.20,-21.3585,55.7679,
25,Ransang,PH,78.89,8.9603,117.5828,
27,Diffa,NE,89.35,13.3154,12.6113,
28,Rikitea,PF,77.49,-23.1203,-134.9692,
33,Cabo San Lucas,MX,82.99,22.8909,-109.9124,
37,Valera,VE,86.97,9.3178,-70.6036,


In [16]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [19]:
# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    # Get lat & lng
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add lat & lng to loc key for params dict
    params["location"] = f"{lat},{lng}"
    
    # Use search term: "lodging" and lat & lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get JSON data from search
    hotels = requests.get(base_url, params=params).json()
    
    # Grab first hotel from results and store name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
hotel_df.head(20)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Kapaa,US,77.00,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
14,Hilo,US,80.60,19.7297,-155.0900,Hilo Hawaiian Hotel
17,Pontianak,ID,75.20,-0.0333,109.3333,Aston Pontianak Hotel & Convention Center
22,Atuona,PF,80.51,-9.8000,-139.0333,Villa Enata
23,Saint-Philippe,RE,75.20,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
25,Ransang,PH,78.89,8.9603,117.5828,Black and Crystal Sand Campsite
27,Diffa,NE,89.35,13.3154,12.6113,ProDAF/Diffa
28,Rikitea,PF,77.49,-23.1203,-134.9692,Pension Maro'i
33,Cabo San Lucas,MX,82.99,22.8909,-109.9124,Hotel Tesoro Los Cabos
37,Valera,VE,86.97,9.3178,-70.6036,Hotel Camino Real


In [28]:
# Add a heatmap of temperature for vacation spots
locations = hotel_df[["Lat", "Lng"]]
max_temps = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, 
             max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call figure to plot data
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# Add heatmap of temperature for vacation spots and marker for each city
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
                                dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call figure to plot data
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [32]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [33]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))